# **06 Data Cleaning and Feature Engineering**

## Objectives

* clean data and engineer features for modelling
- ETL: Impute/handle missing values
- ETL: Treat outliers
- ETL: Encode categoricals
- ETL: Scale/normalise features
- ETL: Engineer domain features
- ETL: Save cleaned dataset

## Inputs

* Merged housing data sources data file: ppd_with_geography_and_imd_epc.csv

## Outputs

* housing.csv

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# - ETL: Impute/handle missing values

Load libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.pipeline import Pipeline

In [ ]:
# load the data
df = pd.read_csv('../data/clean/ppd_with_geography_and_imd_epc.csv')
df.head()

---

In [ ]:
df.info()

# Handle missing values

In [ ]:
# show number of missing values in each column
df.isnull().sum()


In [ ]:
# check for missing floor area values
df['total_floor_area'].isnull().sum()
# delete rows for missing floor area values
df = df[df['total_floor_area'].notnull()]
df.info()


---

In [ ]:
# check data types
df.dtypes
# summary statistics
df.describe()
# check for duplicates
df.duplicated().sum()


In [ ]:
"""
drop the columns that are not needed for analysis
Columns marked with * are to be dropped
0 transaction 7569 non-null object
1 price 7569 non-null int64  
2 transfer_date 7569 non-null object  
3 postcode 7569 non-null object  
4 property_type 7569 non-null object  
5 new_build 7569 non-null object  
6 tenure 7569 non-null object  
7 PAON 7569 non-null object  
8 SAON 428 non-null object  
9 Street 7531 non-null object  
10 Locality 3163 non-null object  
11 town_city 7569 non-null object  
12 district 7569 non-null object  
13 county 7569 non-null object  
*14 PPD_category 7569 non-null object  
*15 Status 7569 non-null object  
16 lsoa11cd 7569 non-null object  
17 msoa11nm 7569 non-null object  
18 ladnm 7569 non-null object  
19 IMD_Decile 7569 non-null float64  
20 IMD_Rank 7569 non-null float64  
*21 addr_key 7569 non-null object  
*22 exact_key_no_saon 7569 non-null object  
23 current_energy_rating 7569 non-null object  
24 total_floor_area 7569 non-null float64  
*25 matched_key 3249 non-null object  
*26 match_score 3249 non-null float64
"""
# drop the columns that are not needed for analysis 
df = df.drop(columns=['PPD_category', 'Status', 'exact_key_no_saon', 'matched_key', 'match_score'])
df.info()

Check if any properties have had more than one transaction in the PPD data

In [ ]:
# is addr_key unique?
df['addr_key'].is_unique
# are there duplicate addr_key values?
df['addr_key'].duplicated().sum()
# what are the duplicate addr_key values?
df[df['addr_key'].duplicated(keep=False)].sort_values('addr_key')
# keep only the last date for each addr_key
df = df.sort_values('transfer_date').drop_duplicates('addr_key', keep='last')

In [ ]:
# date conversion
df['transfer_date'] = pd.to_datetime(df['transfer_date'], errors='coerce')
df['year'] = df['transfer_date'].dt.year

In [ ]:
# Use data profiling using ydata_profiling to generate a report to identify any other data quality issues
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_file("../reports/data_profiling_report.html")
# check for outliers

**Considerations from the profile report**
alerts in the profile report:

IMD_Decile and IMD_Rank: High correlation
Action: keep only IMD_Decile as a feature

current_energy_rating and new_build: High correlation
New builds usually have a higher EPC.
Action: create an interaction (new_build × energy_band_num) 
new_build needs to be a numeric flag

price and total_floor_area: High correlation
As expected, no action

property_type and tenure: High correlation
Flats are often leasehold
Action: create an interaction flag: is_flat & is_leasehold 

price is highly skewed 
Action: Model log1p(price) as the target (and inverse-transform for reporting).
Action: remove outliers, clipping by property_type


In [ ]:
# Energy band A is best, G is worst
# create a new numeric feature energy_band_num where A=7, B=6, C=5, D=4, E=3, F=2, G=1
df['energy_band_num'] = df['current_energy_rating'].map({
    'A': 7,
    'B': 6,
    'C': 5,
    'D': 4,
    'E': 3,
    'F': 2,
    'G': 1
 })

# plot the distribution of energy_band_num
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='energy_band_num')
plt.title('Distribution of Energy Bands')
plt.xlabel('Energy Band Letter and Number')
plt.ylabel('Count')
plt.xticks(ticks=range(7), labels=['A: 7', 'B: 6', 'C: 5', 'D: 4', 'E: 3', 'F: 2', 'G: 1'])
plt.show()


In [ ]:
# new_build needs to be a numeric flag
# Action: create an interaction (is_new_build_energy_band_num) 
df['is_new_build'] = df['new_build'].map({'Y': 1, 'N': 0}).astype(int)
df['is_new_build'].value_counts()
df['is_new_build_energy_band_num'] = df['is_new_build'] * df['energy_band_num']
df.info()

In [ ]:
# create an outcode feature from the postcode column, this may be a useful geographic feature
df['outcode'] = df['postcode'].str.extract(r'^([A-Z]{1,2}\d{1,2}[A-Z]?)')
df['outcode'] = df['outcode'].str.strip()

In [ ]:
# Handle outliers in property price 
# plot boxplots to visualize price outliers
plt.figure(figsize=(10, 6))
sns.boxplot(data=df['price'])
plt.title('Property Price Outliers')
plt.show()


In [ ]:
# remove any properties with extreme prices
# calculate the 1st and 99th percentiles
q_low = df['price'].quantile(0.01)
q_high = df['price'].quantile(0.99)
df = df[(df['price'] > q_low) & (df['price'] < q_high)]
df.info()

In [ ]:
# Histogram of price
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='price', bins=30, kde=True)
plt.title('Distribution of Property Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# This is a clearly a skewed distribution so log transform
df = df.copy()
df['log_price'] = np.log1p(df['price'])

In [ ]:
# distribution of log_price
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='log_price', bins=30, kde=True)
plt.title('Distribution of Log-Transformed Property Prices')
plt.xlabel('Log Price')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# This is now less skewed and more normally distributed
# calculate skewness and kurtosis
skewness = df['log_price'].skew()
kurtosis = df['log_price'].kurtosis()
print(f'Skewness: {skewness}, Kurtosis: {kurtosis}')

In [ ]:
# box plot of total_floor_area
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, y='total_floor_area')
plt.title('Box Plot of Total Floor Area')
plt.xlabel('Total Floor Area')
plt.show()


In [ ]:
# There are some extreme outliers in total_floor_area
# remove outliers outside the 1st and 99th percentiles
q_low = df['total_floor_area'].quantile(0.01)
q_high = df['total_floor_area'].quantile(0.99)
df = df[(df['total_floor_area'] > q_low) & (df['total_floor_area'] < q_high)]
df.info()

In [ ]:
# distribution of total_floor_area
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='total_floor_area', bins=30, kde=True)
plt.title('Distribution of Total Floor Area')
plt.xlabel('Total Floor Area')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# log transform total_floor_area to reduce skewness
df['log_total_floor_area'] = np.log1p(df['total_floor_area'])
# distribution of log_total_floor_area
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='log_total_floor_area', bins=30, kde=True)
plt.title('Distribution of Log-Transformed Total Floor Area')
plt.xlabel('Log Total Floor Area')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# skewness and kurtosis of log_total_floor_area
skewness = df['log_total_floor_area'].skew()
kurtosis = df['log_total_floor_area'].kurtosis()
print(f'Skewness: {skewness}, Kurtosis: {kurtosis}')

In [ ]:
# plot scatterplot of log_price vs log_total_floor_area 
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='log_total_floor_area', y='log_price')
plt.title('Scatter Plot of Log Total Floor Area vs Log Price')
plt.xlabel('Log Total Floor Area')
plt.ylabel('Log Price')
plt.show()


In [ ]:
# plot the property_type and tenure 
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='property_type', hue='tenure')
plt.title('Property Type by Tenure')
plt.xlabel('Property Type')
plt.ylabel('Count')
plt.show()

NOTE

Clearly, the vast majority of flats are leasehold.
Action: create an interaction flag: is_flat & is_leasehold 

In [ ]:
# Action: create an interaction flag: is_flat & is_leasehold 
df['is_flat'] = df['property_type'] == 'Flat'
df['is_leasehold'] = df['tenure'] == 'Leasehold'
df['is_flat_leasehold'] = df['is_flat'] & df['is_leasehold']
df.info()

In [ ]:
# convert new_build to boolean
df['new_build'] = df['new_build'].map({'Y': True, 'N': False})
df['new_build'].value_counts()

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---